# Data Loading

Loading the SkillCorner A-League tracking data. Need to understand the structure before jumping into analysis.

Files per match:
- `{id}_match.json` - metadata, lineups, pitch dimensions
- `{id}_tracking_extrapolated.jsonl` - 10fps tracking data
- `{id}_dynamic_events.csv` - pre-computed events
- `{id}_phases_of_play.csv` - game phases

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import json
from pathlib import Path

from src.loaders import (
    load_match_metadata,
    load_tracking_data,
    load_dynamic_events,
    load_phases,
    get_all_match_ids,
    load_all_matches
)

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("✓ Imports successful")

✓ Imports successful


## 1: Explore One Match First

Let's load match 1886347 to see what we're working with.

In [2]:
match_id = "1886347"
metadata = load_match_metadata(match_id)

# Basic match info
print(f"Match: {metadata['home_team']['name']} vs {metadata['away_team']['name']}")
print(f"Date: {metadata['date_time']}")
print(f"Score: {metadata['home_team_score']}-{metadata['away_team_score']}")
print(f"Pitch: {metadata['pitch_length']}m x {metadata['pitch_width']}m")
print(f"Players: {len(metadata['players'])}")

Match: Auckland FC vs Newcastle United Jets FC
Date: 2024-11-30T04:00:00Z
Score: 2-0
Pitch: 104m x 68m
Players: 36


In [3]:
# Check player structure
print("Sample player object:")
print(json.dumps(metadata['players'][0], indent=2))

Sample player object:
{
  "player_role": {
    "id": 15,
    "position_group": "Center Forward",
    "name": "Center Forward",
    "acronym": "CF"
  },
  "start_time": "00:00:00",
  "end_time": "01:25:21",
  "number": 10,
  "yellow_card": 0,
  "red_card": 0,
  "injured": false,
  "goal": 0,
  "own_goal": 0,
  "playing_time": {
    "total": {
      "minutes_tip": 29.55,
      "minutes_otip": 18.76,
      "start_frame": 10,
      "end_frame": 52009,
      "minutes_played": 86.65,
      "minutes_played_regular_time": 86.65
    },
    "by_period": [
      {
        "name": "period_1",
        "minutes_tip": 18.21,
        "minutes_otip": 11.03,
        "start_frame": 10,
        "end_frame": 27790,
        "minutes_played": 46.3
      },
      {
        "name": "period_2",
        "minutes_tip": 11.34,
        "minutes_otip": 7.73,
        "start_frame": 27800,
        "end_frame": 52009,
        "minutes_played": 40.35
      }
    ]
  },
  "team_player_id": 1507965,
  "team_id": 4177,
  "

### 1a: Tracking Data

This is the big one - 10fps position data for players and ball.

In [4]:
tracking = load_tracking_data(match_id)

print(f"Shape: {tracking.shape}")
print(f"Memory: {tracking.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"\nColumns: {list(tracking.columns)}")

Shape: (59061, 7)
Memory: 50.3 MB

Columns: ['frame', 'timestamp', 'period', 'ball_data', 'possession', 'image_corners_projection', 'player_data']


In [5]:
# Look at frames with actual match data (first frames are empty pre-match)
print("\n\nFrames 10-15 (actual match data):")
tracking.iloc[10:15]



Frames 10-15 (actual match data):


,frame,timestamp,period,ball_data,possession,image_corners_projection,player_data
10,10,2025-12-10 00:00:00.000,1.0,"{'x': 0.32, 'y': 0.38, 'z': 0.13, 'is_detected...","{'player_id': None, 'group': None}","{'x_top_left': -52.52, 'y_top_left': 39.0, 'x_...","[{'x': -39.63, 'y': -0.08, 'player_id': 51009,..."
11,11,2025-12-10 00:00:00.100,1.0,"{'x': 0.54, 'y': 0.08, 'z': 0.22, 'is_detected...","{'player_id': None, 'group': None}","{'x_top_left': -52.37, 'y_top_left': 39.0, 'x_...","[{'x': -39.86, 'y': -0.13, 'player_id': 51009,..."
12,12,2025-12-10 00:00:00.200,1.0,"{'x': 0.5700000000000001, 'y': -0.07, 'z': 0.1...","{'player_id': None, 'group': None}","{'x_top_left': -52.11, 'y_top_left': 39.0, 'x_...","[{'x': -40.06, 'y': -0.18, 'player_id': 51009,..."
13,13,2025-12-10 00:00:00.300,1.0,"{'x': 0.56, 'y': -0.07, 'z': 0.14, 'is_detecte...","{'player_id': None, 'group': None}","{'x_top_left': -52.02, 'y_top_left': 39.0, 'x_...","[{'x': -40.24, 'y': -0.22, 'player_id': 51009,..."
14,14,2025-12-10 00:00:00.400,1.0,"{'x': 0.59, 'y': -0.03, 'z': 0.14, 'is_detecte...","{'player_id': None, 'group': None}","{'x_top_left': -51.78, 'y_top_left': 39.0, 'x_...","[{'x': -40.39, 'y': -0.25, 'player_id': 51009,..."


In [6]:
# The nested structures - ball_data, possession, player_data are stored as objects
# Will need to flatten these for analysis

# Let's grab the first frame where a player has possession
# Then, we can check the player_data and ball_data for that possession

# First frame where someone actually has the ball
frame_with_possession = tracking[tracking["possession"].apply(
    lambda x: x.get("player_id") is not None if isinstance(x, dict) else False
)].iloc[0]

print("\nFirst possession in frame:")
print(frame_with_possession["possession"])

# Pull out the ball + the player who owns it in the above frame
ball = frame_with_possession["ball_data"]
possessing_player = next(
    p for p in frame_with_possession["player_data"]
    if p["player_id"] == frame_with_possession["possession"]["player_id"]
)

print("\nPlayer and ball at that frame:")
print("possessing player", possessing_player)
print("ball_position:", ball)  # matches the sample ball_data structure


First possession in frame:
{'player_id': 966120, 'group': 'away team'}

Player and ball at that frame:
possessing player {'x': 0.73, 'y': 0.49, 'player_id': 966120, 'is_detected': True}
ball_position: {'x': 0.03, 'y': -0.36, 'z': 0.33, 'is_detected': True}


In [7]:
# How many players per frame when match is active?
active_frames = tracking[tracking['period'].notna()]
players_per_frame = active_frames['player_data'].apply(lambda x: len(x) if isinstance(x, list) else 0)

print(f"\nPlayers per frame during active play:")
print(players_per_frame.describe())
print(f"Most common: {players_per_frame.mode()[0]} players per frame")


Players per frame during active play:
count    59042.000000
mean        16.193151
std          9.697050
min          0.000000
25%          0.000000
50%         22.000000
75%         22.000000
max         22.000000
Name: player_data, dtype: float64
Most common: 22 players per frame


**Note**: The tracking data has nested lists/dicts. For player-level metrics, I'll need to explode player_data into individual rows. Each frame has ~22 players tracked.

### 1b: Dynamic Events

The dynamic_events data is a CSV. Each row corresponds to a specific event_id belonging to 4 subcategories. Note:
* an event_id is unique to a game only
* the x/y attributes of each event are not scaled to standard pitchsize and require adjustement


In [8]:
events = load_dynamic_events(match_id)

print(f"Shape: {events.shape}")
print(f"Memory: {events.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"\n{events.shape[1]} columns found")

Shape: (5079, 294)
Memory: 29.2 MB

294 columns found


Given there are almost 300 columns, we'll need to assess which columns are the most useful for exploration. Fortunately, SkillCorner have provided some great documentation that helps us understand these better.

### 1c: What SkillCorner Already Computed  
*(Taken directly from the **Dynamic Events** PDF provided in the GitHub repo.)*

The events CSV has 294 pre-flattened columns because each event type comes with its own model outputs and contextual metrics. Here’s what is already computed:

**4 event types:**

1. **player_possession** — Every time a player controls the ball  
   - Includes *xloss* and *xshot* predictions (from their progression model)  
   - Aggregates all *passing_options* and *off_ball_runs* that occurred during the possession window  

2. **passing_option** — Potential pass targets detected by their Receiver Model  
   - *passing_option_score*: probability this player becomes the target (GNN-based)  
   - *xthreat*: probability of scoring in the next 10 seconds if the pass completes  
   - *xpass_completion*: probability the pass completes  
   - *served*: True if the pass was made, False if the opportunity wasn’t taken  

3. **off_ball_run** — Attacking movements without the ball (10 subtypes: behind, overlap, etc.)  
   - Pre-computed *speed_avg* and speed-band classifications  
   - *xthreat* for evaluating danger created  
   - *served* indicates whether the run received the ball  

4. **on_ball_engagement** — Defensive actions (pressing, pressure, counter-press, etc.)  
   - EPV outputs such as *stop_possession_danger* and *reduce_possession_danger*  
   - Indicators for which defensive line is threatened  
   - Detected using their GNN–LSTM architecture  

Every event row is also linked to its corresponding **phase_of_play** for tactical context.

**Bottom line:** The heavy modelling is already done. The task is to aggregate, interpret, and combine these outputs — not to re-engineer SkillCorner’s models.

In [9]:
# Based on the columns, let's look at the core identification and temporal fields
key_cols = ['event_id', 'event_type', 'event_subtype', 'player_id', 'player_name', 
            'frame_start', 'frame_end', 'time_start', 'time_end', 'x_start', 'x_end',
            'y_start', 'y_end']

print("Sample events (key columns):")
events[key_cols].head(10)

Sample events (key columns):


,event_id,event_type,event_subtype,player_id,player_name,frame_start,frame_end,time_start,time_end,x_start,x_end,y_start,y_end
0,8_0,player_possession,NaN,966120,B. Gibson,28,28,00:01.8,00:01.8,0.73,0.73,0.49,0.49
1,8_1,player_possession,NaN,51649,A. Šušnjar,48,58,00:03.8,00:04.8,-22.31,-22.21,1.22,2.70
2,7_0,passing_option,NaN,735574,K. Grozos,48,53,00:03.8,00:04.3,-10.47,-11.36,-2.78,-1.38
3,7_1,passing_option,NaN,735578,M. Natta,48,58,00:03.8,00:04.8,-20.69,-20.36,16.66,17.69
4,9_0,on_ball_engagement,pressing,50951,J. Brimmer,56,58,00:02.4,00:04.8,13.03,13.98,0.06,-0.68
5,8_2,player_possession,NaN,735578,M. Natta,72,89,00:06.2,00:07.9,-17.60,-10.75,20.17,24.18
6,7_2,passing_option,NaN,50978,C. Timmins,72,82,00:06.2,00:07.2,-4.66,-6.39,11.04,14.27
7,7_3,passing_option,NaN,795507,L. Bayliss,72,89,00:06.2,00:07.9,14.93,11.67,10.13,16.14
8,7_4,passing_option,NaN,966120,B. Gibson,72,89,00:06.2,00:07.9,12.73,16.91,3.61,9.27
9,7_5,passing_option,NaN,735573,T. Aquilina,73,89,00:06.3,00:07.9,7.31,6.87,19.98,27.64


In [10]:
# What event types do we have?
print("Event type distribution:")
print(events['event_type'].value_counts())

Event type distribution:
event_type
passing_option        2544
player_possession      999
on_ball_engagement     937
off_ball_run           599
Name: count, dtype: int64


In [11]:
# Quick check: what subtypes exist?
print("Event subtypes by type:")
for event_type in events['event_type'].unique():
    mask = events['event_type'] == event_type
    subtypes = events[mask]['event_subtype'].dropna().unique()
    
    print(f"\n{event_type}:")
    if len(subtypes) > 0:
        print(f"  {', '.join(subtypes)}")
    else:
        print("  (no subtypes)")

Event subtypes by type:

player_possession:
  (no subtypes)

passing_option:
  (no subtypes)

on_ball_engagement:
  pressing, recovery_press, pressure, counter_press, other

off_ball_run:
  pulling_wide, coming_short, behind, support, overlap, cross_receiver, dropping_off, run_ahead_of_the_ball, pulling_half_space, underlap


In [12]:
# Lots of sparse columns (expected - different events have different fields)
null_pct = (events.isnull().sum() / len(events) * 100)
print(f"Columns with >90% nulls: {(null_pct > 90).sum()}")
print("\nThis is normal - most columns only apply to specific event types")

Columns with >90% nulls: 27

This is normal - most columns only apply to specific event types


### 1d: Phases of Play

The phase of play data is CSV. Each row corresponds to the start and end frames of a given phase
* Phases of play capture which phase the attacking and defending team are in concurrently.
* Phases of play are only defined when the ball is in play. When the ball is out of play there is no phase of play
* Each in-possession phase directly corresponds to an out-of-possession phase.


In [13]:
phases = load_phases(match_id)

print(f"Shape: {phases.shape}")
phases.head()

Shape: (454, 44)


,index,match_id,frame_start,frame_end,time_start,time_end,minute_start,second_start,duration,period,attacking_side_id,team_in_possession_id,attacking_side,team_in_possession_shortname,n_player_possessions_in_phase,team_possession_loss_in_phase,team_possession_lead_to_goal,team_possession_lead_to_shot,team_in_possession_phase_type,team_in_possession_phase_type_id,team_out_of_possession_phase_type,team_out_of_possession_phase_type_id,x_start,y_start,channel_id_start,channel_start,third_id_start,third_start,penalty_area_start,x_end,y_end,channel_id_end,channel_end,third_id_end,third_end,penalty_area_end,team_in_possession_width_start,team_in_possession_width_end,team_in_possession_length_start,team_in_possession_length_end,team_out_of_possession_width_start,team_out_of_possession_width_end,team_out_of_possession_length_start,team_out_of_possession_length_end
0,0,1886347,28,89,00:01.8,00:07.9,0,1,6.1,1,1,1805,left_to_right,Newcastle,3,False,False,False,create,1,medium_block,9,0.03,-0.36,3,center,2,middle_third,False,-10.70,23.82,1,wide_left,2,middle_third,False,51.72,54.41,41.44,58.03,34.37,32.81,42.01,52.35
1,1,1886347,89,107,00:07.9,00:09.7,0,7,1.8,1,1,1805,left_to_right,Newcastle,1,True,False,False,direct,6,defending_direct,15,-9.87,23.99,1,wide_left,2,middle_third,False,21.73,16.53,2,half_space_left,3,attacking_third,False,54.72,54.54,58.24,62.34,32.97,34.26,51.98,46.99
2,2,1886347,185,232,00:17.5,00:22.2,0,17,4.7,1,2,4177,right_to_left,Auckland FC,2,False,False,False,build_up,0,high_block,10,-22.52,-7.98,3,center,1,defensive_third,False,-20.40,17.83,2,half_space_left,1,defensive_third,False,45.30,50.78,48.51,50.31,31.40,36.38,54.72,51.13
3,3,1886347,232,283,00:22.2,00:27.3,0,22,5.1,1,2,4177,right_to_left,Auckland FC,1,True,False,False,create,1,medium_block,9,-19.24,19.52,2,half_space_left,1,defensive_third,False,29.20,26.16,1,wide_left,3,attacking_third,False,50.81,42.83,50.33,63.89,36.59,38.73,51.22,53.46
4,4,1886347,283,301,00:27.3,00:29.1,0,27,1.8,1,1,1805,left_to_right,Newcastle,1,True,False,False,chaotic,5,chaotic,5,-29.73,-26.19,5,wide_right,1,defensive_third,False,-36.35,-29.24,5,wide_right,1,defensive_third,False,38.61,36.66,53.59,54.36,42.62,40.13,64.54,71.83


In [14]:
# Phase types
print(phases['team_in_possession_phase_type'].value_counts())

team_in_possession_phase_type
create         156
chaotic         97
finish          89
build_up        63
direct          23
set_play        20
transition       3
quick_break      3
Name: count, dtype: int64


## 2: Load All 10 Matches

In [15]:
match_ids = get_all_match_ids()
print(f"Found {len(match_ids)} matches: {match_ids}")

Found 10 matches: ['2017461', '2015213', '2013725', '2011166', '2006229', '1996435', '1953632', '1925299', '1899585', '1886347']


In [16]:
# This takes ~10-20 seconds
all_data = load_all_matches()
print(f"\n✓ Loaded {len(all_data)} matches")

INFO - Loading match 1/10: 2017461
INFO - Loading match 2/10: 2015213
INFO - Loading match 3/10: 2013725
INFO - Loading match 4/10: 2011166
INFO - Loading match 5/10: 2006229
INFO - Loading match 6/10: 1996435
INFO - Loading match 7/10: 1953632
INFO - Loading match 8/10: 1925299
INFO - Loading match 9/10: 1899585
INFO - Loading match 10/10: 1886347
INFO - Loaded 10/10 matches



✓ Loaded 10 matches


In [17]:
# Quick summary
for match_id, data in all_data.items():
    meta = data['metadata']
    print(f"{match_id}: {meta['home_team']['name'][:15]:15s} vs {meta['away_team']['name'][:15]:15s} | "
          f"Frames: {len(data['tracking']):6,} | Events: {len(data['events']):4,}")

2017461: Melbourne Victo vs Auckland FC     | Frames: 71,451 | Events: 4,188
2015213: Western United  vs Auckland FC     | Frames: 72,101 | Events: 4,582
2013725: Western United  vs Sydney Football | Frames: 70,251 | Events: 4,999
2011166: Wellington Phoe vs Melbourne Victo | Frames: 71,851 | Events: 3,966
2006229: Melbourne City  vs Macarthur FC    | Frames: 59,270 | Events: 4,991
1996435: Sydney Football vs Adelaide United | Frames: 57,621 | Events: 5,292
1953632: Central Coast M vs Melbourne City  | Frames: 59,250 | Events: 4,823
1925299: Brisbane Roar F vs Perth Glory Foo | Frames: 61,301 | Events: 5,220
1899585: Auckland FC     vs Wellington Phoe | Frames: 60,530 | Events: 4,713
1886347: Auckland FC     vs Newcastle Unite | Frames: 59,061 | Events: 5,079


## 3: Quick Data Quality Checks

SkillCorner’s data is generally reliable, so I’m not trying to re-audit their whole pipeline. 

These checks are just the bare minimum I need to trust my own work. 

If a match is missing events, phases, tracking, or metadata, or if core event columns have nulls, everything downstream falls apart. Same deal with obviously broken match durations. 

So this step isn’t about questioning SkillCorner — it’s about making sure the assumptions my analysis depends on actually hold before I build anything on top of it.

### 3a: Match metadata completeness

In [18]:
print("Checking data completeness across matches...")
missing = []
for match_id, data in all_data.items():
    required_keys = ['metadata', 'events', 'phases', 'tracking']
    if not all(k in data for k in required_keys):
        missing.append(match_id)

if missing:
    print(f"WARNING: {len(missing)} matches missing data: {missing}")
else:
    print(f"All {len(all_data)} matches have metadata, events, phases and tracking\n")

Checking data completeness across matches...
All 10 matches have metadata, events, phases and tracking



### 3b: Null checks

In [19]:
print("Checking for nulls in core event columns...")
all_events_temp = pd.concat(
    [
        d['events'][['match_id', 'frame_start', 'frame_end', 'event_type']]
        for d in all_data.values()
    ],
    ignore_index=True,
)
null_counts = all_events_temp.isnull().sum()

if (null_counts > 0).any():
    print("Null values found:")
    print(null_counts[null_counts > 0])
else:
    print("No nulls in core event columns\n")

Checking for nulls in core event columns...
No nulls in core event columns



### 3c: Duration checks

In [20]:
print("Approximate match durations (minutes) from tracking frames:")
for match_id, data in all_data.items():
    frames = data['tracking']['frame']
    duration = (frames.max() - frames.min()) / 600.0  # 10 fps -> 600 frames per minute
    print(f"  {match_id}: {duration:.1f} min")

Approximate match durations (minutes) from tracking frames:
  2017461: 119.1 min
  2015213: 120.2 min
  2013725: 117.1 min
  2011166: 119.8 min
  2006229: 98.8 min
  1996435: 96.0 min
  1953632: 98.7 min
  1925299: 102.2 min
  1899585: 100.9 min
  1886347: 98.4 min


### 3d: Player count

In [21]:
# Player counts per match (expect ~30-36 with subs)
player_counts = {mid: len(d['metadata']['players']) for mid, d in all_data.items()}
print("Player counts per match:")
print(pd.Series(player_counts).describe())

Player counts per match:
count    10.0
mean     36.0
std       0.0
min      36.0
25%      36.0
50%      36.0
75%      36.0
max      36.0
dtype: float64


### 3e: Tracking frame count

In [22]:
# Tracking frame counts (expect ~54k for 90min match at 10fps)
frame_counts = {mid: len(d['tracking']) for mid, d in all_data.items()}
print("\nTracking frame counts:")
print(pd.Series(frame_counts).describe())


Tracking frame counts:
count       10.000000
mean     64268.700000
std       6240.379886
min      57621.000000
25%      59255.000000
50%      60915.500000
75%      71151.000000
max      72101.000000
dtype: float64


### 3f: Memory calc

In [23]:
# Total memory usage
total_mb = sum(
    d['tracking'].memory_usage(deep=True).sum() +
    d['events'].memory_usage(deep=True).sum() +
    d['phases'].memory_usage(deep=True).sum()
    for d in all_data.values()
) / 1024 / 1024

print(f"\nTotal memory: {total_mb:.1f} MB (~{total_mb/len(all_data):.1f} MB per match)")
print("Manageable in-memory for 10 matches")

# Optional: show breakdown by data type
tracking_mb = sum(d['tracking'].memory_usage(deep=True).sum() for d in all_data.values()) / 1024 / 1024
events_mb = sum(d['events'].memory_usage(deep=True).sum() for d in all_data.values()) / 1024 / 1024
phases_mb = sum(d['phases'].memory_usage(deep=True).sum() for d in all_data.values()) / 1024 / 1024

print(f"\nBreakdown:")
print(f"  Tracking: {tracking_mb:.1f} MB ({tracking_mb/total_mb*100:.0f}%)")
print(f"  Events: {events_mb:.1f} MB ({events_mb/total_mb*100:.0f}%)")
print(f"  Phases: {phases_mb:.1f} MB ({phases_mb/total_mb*100:.0f}%)")


Total memory: 819.8 MB (~82.0 MB per match)
Manageable in-memory for 10 matches

Breakdown:
  Tracking: 540.9 MB (66%)
  Events: 275.3 MB (34%)
  Phases: 3.6 MB (0%)


## 4: Next Steps

Data successfully loaded. Key observations:

**Tracking data (frame-level):**
- 10 matches, ~22 players per active frame, nested player_data requires exploding
- Coordinates: pitch center origin (will normalize using pitch_length/width for cross-match analysis)
- Can calculate speeds from x/y for sprint detection
- Some frames have null periods (stoppages - expected)

**Events data (pre-computed ML metrics):**
- 4 event types with metrics already computed:
  - xthreat (Threat Model), passing_option_score (Receiver Model), xpass_completion (completion model)
  - Line breaks detected with defensive structure identification
  - Speed bands already calculated for event-based movements
- Phase of play fields attached to every event

**What this means:**
- Calculate sprints from tracking (demonstrates raw data handling, matches Physical Glossary standards)
- Aggregate events using pre-computed metrics (no need to rebuild ML models)
- Join to phases for tactical context (enables "sprints by phase", "runs by phase")

**Still need:**
- Load phases_of_play.csv in EDA notebook (contextual data, not primary structure)
- Will use for phase-aware metric splits

Ready for EDA and metric aggregation.